In [1]:
%matplotlib widget
import ipympl
import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import display
from scipy.interpolate import InterpolatedUnivariateSpline
from numpy import linalg as LA
### Import libraries to be used throughout
# basic psi4 library
import psi4
# numpy
import numpy as np
# scipy
from scipy.interpolate import InterpolatedUnivariateSpline
# linear algebra package from numpy
from numpy import linalg as LA
# time-dependent scf library from psi4 for computing excited states and transition dipole moments
from psi4.driver.procrouting.response.scf_response import tdscf_excitations
#from matplotlib import pyplot as plt

In [2]:
# Create a slider widget and a widget to hold the plot.
slider = widgets.FloatSlider(description = r'$g$',
                            value = 0,
                            min = 0,
                            max = 5,  
                            step = 1)
# We can use HTML styles to control the appearance of the widget.
plot_widget = widgets.Output(layout = {'width':'100%', 'border': '1px solid black'})

In [3]:
# function to plot
def lp(amplitude, omega, r_val, g_spline, e_spline, tdm_spline):
    H = np.zeros((3,3))
    lp_vals = np.zeros_like(r_val)
    for i in range(0, len(r_val)):
        H[0,0] = g_spline(r_val[i])
        H[1,1] = g_spline(r_val[i]) + omega
        H[1,2] = amplitude * tdm_spline(r_val[i])
        H[2,1] = amplitude * tdm_spline(r_val[i])
        H[2,2] = e_spline(r_val[i])
        vals, vecs = LA.eigh(H)
        lp_vals[i] = vals[1]
        
    return lp_vals


# function to plot
def up(amplitude, omega, r_val, g_spline, e_spline, tdm_spline):
    H = np.zeros((3,3))
    up_vals = np.zeros_like(r_val)
    for i in range(0, len(r_val)):
        H[0,0] = g_spline(r_val[i])
        H[1,1] = g_spline(r_val[i]) + omega
        H[1,2] = amplitude * tdm_spline(r_val[i])
        H[2,1] = amplitude * tdm_spline(r_val[i])
        H[2,2] = e_spline(r_val[i])
        vals, vecs = LA.eigh(H)
        up_vals[i] = vals[2]
        
    return up_vals

## create an array of x values
#x = np.linspace(-np.pi, np.pi, 201)
rs = np.array([1.3, 1.42, 1.54, 
               1.6600000000000001, 1.78, 1.9, 
               2.02, 2.14, 2.26, 2.38, 2.5, 
               2.62, 2.74, 2.8600000000000003, 
               2.98, 3.0999999999999996, 3.2199999999999998, 
               3.34, 3.46, 3.58])

Eg = np.array([-200.34317863, -200.36733437, -200.3788335,  -200.38211913, -200.38005015,
 -200.37454644, -200.36693436, -200.35813969, -200.34880828, -200.33938762,
 -200.33018284, -200.32139619, -200.3131552,  -200.30553287, -200.29856231,
 -200.29224756, -200.28657217, -200.2815057,  -200.27700922, -200.27303916])

Ee = np.array([-200.11946659, -200.15343212, -200.17455494, -200.18699014, -200.19340378,
 -200.19564856, -200.19506567, -200.19266834, -200.18922925, -200.18533058,
 -200.18141088, -200.17778616, -200.17467719, -200.17223017, -200.17053044,
 -200.16961006, -200.16945866, -200.17002824, -200.17124271, -200.17300921])

mu = np.array([1.41026992, 1.46500868, 1.53226812, 1.60873137, 1.69113032, 1.77658259,
 1.86280274, 1.9479899,  2.03067753, 2.10965627, 2.18375603, 2.25178818,
 2.31244456, 2.36432877, 2.40606639, 2.43656128, 2.45526201, 2.46232396,
 2.45859393, 2.44546058])

lp_array = np.array([-200.18542568, -200.20965361, -200.22126158, -200.22471227, -200.2228985,
 -200.21780423, -200.2108843,  -200.20332124, -200.19614803, -200.18997948,
 -200.18480472, -200.18045717, -200.17688908, -200.17412067, -200.17217721,
 -200.17106054, -200.17074442, -200.17117242, -200.17226379, -200.17392288])

up_array = np.array([-200.11919521, -200.15308855, -200.17410253, -200.18637267, -200.1925311,
 -200.19436644, -200.1930914,  -200.18946246, -200.18386517, -200.17671439,
 -200.16876468, -200.16070086, -200.15291899, -200.14561803, -200.13889122,
 -200.13277276, -200.12726208, -200.12233719, -200.11796381, -200.11410116])


#a_spline = InterpolatedUnivariateSpline(rs, lp, k=3)
#b_spline = InterpolatedUnivariateSpline(rs, up, k=3)
Eg_spline = InterpolatedUnivariateSpline(rs, Eg, k=3)
Ee_spline = InterpolatedUnivariateSpline(rs, Ee, k=3)
mu_spline = InterpolatedUnivariateSpline(rs, mu, k=3)

om = 4.3 / 27.211

compute_lp = lp(0.003, om, rs, Eg_spline, Ee_spline, mu_spline)
compute_up = up(0.003, om, rs, Eg_spline, Ee_spline, mu_spline)


assert np.allclose(compute_lp, lp_array)
assert np.allclose(compute_up, up_array)

# function to plot
def a_sin(amplitude, x, a_spline):
    return amplitude * a_spline(x)

def b_sin(amplitude, x, b_spline):
    return amplitude * b_spline(x)

In [4]:

# Turn off interactive mode before creating the 
# plot so it doesn't display too early.
plt.ioff() # Turn off interactive mode
fig, ax = plt.subplots(constrained_layout = True, figsize=[5,4]);
plt.ion() # Turn on interactive mode

#line1, = ax.plot(rs, a_sin(slider.value, rs, a_spline));
#line2, = ax.plot(rs, b_sin(slider.value, rs, b_spline));
line1, = ax.plot(rs, lp(0.00, om, rs, Eg_spline, Ee_spline, mu_spline));
line2, = ax.plot(rs, up(0.00, om, rs, Eg_spline, Ee_spline, mu_spline));
ax.set_ylabel(r'$A\sin(x)$')
ax.set_xlabel(r'$x$')

# force the figure to display in the plot_widget
with plot_widget:
    display(fig.canvas)


In [5]:
# Create a function to re-draw the plot using the value of the slider. It is 
# faster to change the y-data but you can erase the plot and draw a new one.
def update(value):
    '''We can use `slider.value` or `value.new` to get the new slider value.'''
    with plot_widget:
        line1.set_ydata(lp(slider.value*0.001, om, rs, Eg_spline, Ee_spline, mu_spline))
        line2.set_ydata(up(slider.value*0.001, om, rs, Eg_spline, Ee_spline, mu_spline))
        fig.canvas.draw()

# Set an observer to call `update` whenever the value changes.
slider.observe(update)

Use the slider below to change the amplitude, $A$, of the sine function.

In [6]:
widgets.VBox([plot_widget, slider])